# 1. Set up the notebook

Import modules.

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from IPython.display import display, IFrame, HTML

Define a function to load flight data from hardware experiments and resample it at 100 Hz using linear interpolation.

In [ ]:
def load_hardware_data(filename, t_min_offset=0, t_max_offset=0):
    # load raw data
    with open(filename, 'r') as f:
        data = json.load(f)

    # convert lists to numpy arrays
    for val in data.values():
        for key in val.keys():
            val[key] = np.array(val[key])

    # create an array of times at which to subsample
    t_min = -np.inf
    t_max = np.inf
    for key, val in data.items():
        t_min = max(t_min, val['time'][0])
        t_max = min(t_max, val['time'][-1])
    t_min += t_min_offset * 1000
    t_max -= t_max_offset * 1000
    nt = int(1 + np.floor((t_max - t_min) / 10.))
    t = np.arange(0, 10 * nt, 10) / 1000.
    resampled_data = {'time': t}

    # resample raw data with linear interpolation
    for k, v in data.items():
        f = interp1d((v['time'] - t_min) / 1000., v['data'])
        resampled_data[k] = f(t)
        
    # return the resampled data
    return resampled_data

Define a function to embed videos that have been uploaded to [Illinois Mediaspace](https://mediaspace.illinois.edu/).

In [ ]:
def mediaspace(url, width=640, height=397):
    video_id = url.rsplit('/', 1)[-1]
    src = f'https://mediaspace.illinois.edu/embed/secure/iframe/entryId/{video_id}?st=0'
    display(IFrame(src, width=width, height=height))
    display(HTML(f'<a href="{url}" target="_blank">Go to video on Illinois Mediaspace</a>'))

# 2. Define constants

Define the acceleration of gravity in $\text{kg}\cdot\text{m} \;/\; \text{s}^2$:

In [ ]:
g = 9.81

Define the mass of the drone in $\text{kg}$:

In [ ]:
m = 1.00 # <-- FIXME

# 3. Estimate the moment of inertia about the $x$-axis

Define the distance in meters between the axis of rotation and the center of mass:

In [ ]:
r = 1.00 # <-- FIXME

Show a photo of your rig (change the file name or file extension as appropriate):

![x-axis-rig](x-rig.png)

Show a video of the drone swinging on your rig, while you were collecting data (change the url as appropriate):

In [ ]:
mediaspace('https://mediaspace.illinois.edu/media/t/YOUR_VIDEO_ID')

Load and resample data.

The `load_hardware_data` function has two optional arguments:

* `t_min_offset` allows you to discard data at the *start* of the experiment - if its value is `4.`, for example, then the *first* four seconds of data will be discarded
* `t_max_offset` allows you to discard data at the *end* of the experiment - if its value is `4.`, for example, then the *last* four seconds of data will be discarded

You may find these arguments useful, since you will likely find that some of the data at the start and end of your experiment are garbage.

In [ ]:
data = load_hardware_data(
    'x_data.json',
    t_min_offset=0.,
    t_max_offset=0.
)

Parse data to get time and the three components of angular velocity (in radians / second).

In [ ]:
t = data['time']
w_x = np.deg2rad(data['gyro.x'])
w_y = np.deg2rad(data['gyro.y'])
w_z = np.deg2rad(data['gyro.z'])

Plot all three components of angular velocity. You are trying to estimate the moment of inertia about the $x$ axis. The component of angular velocity about this axis should be large, and the components of angular velocity about the other two axes should be negligibly small. It is important to check this.

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(t, w_x, label='w_x')
plt.plot(t, w_y, label='w_y')
plt.plot(t, w_z, label='w_z')
plt.legend()
plt.grid()

You should find that `w_x` is oscillatory. The period is the peak-to-peak time. You could measure the period by hand, but it is easier to automate this process, particularly if you want to average your estimate of the period over many oscillations.

Here is one way to do it:

* Find the index $i_k$ of each peak $k\in\{0, \dotsc, n-1\}$ in your data.
* Find the time $t_k$ at each peak for $k\in\{0, \dotsc, n-1\}$.
* Find the difference $T_k = t_{k+1} - t_k$ between consecutive peak times for $k \in \{0, \dotsc, n-2\}$.
* Find the mean difference: $$\widehat{T} = \dfrac{1}{n-1} \sum_{k=0}^{n-2} T_k.$$ This is an estimate of the oscillation period.

Here is one way to implement this in code, using [scipy.signal.find_peaks](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html):

In [ ]:
# Find the index of each peak (increase "prominence" if you get bad results)
peaks = find_peaks(w_x, prominence=0)
i_peaks = peaks[0]

# Find the time at each peak
t_peaks = t[i_peaks]

# Find w_x at each peak (for visualization)
w_x_peaks = w_x[i_peaks]

# Find the difference between consecutive peak times
#
# Note:
#
#  t_peaks[1:] means t_peaks without the first element
#  t_peaks[:-1] means t_peaks without the last element
#
# So, t_peaks[1:] - t_peaks[:-1] produces the following array:
#
#  [t_peaks[1]-t_peaks[0], t_peaks[0]-t_peaks[1], ...]
#
t_diff = t_peaks[1:] - t_peaks[:-1]

# Find the mean difference as an estimate of the oscillation period
T = np.mean(t_diff)

Sanity check - print the peak times:

In [ ]:
print(t_peaks)

Sanity check - plot the peaks:

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(t, w_x, label='w_x')
plt.plot(t_peaks, w_x_peaks, '.', markersize=12)
plt.legend()
plt.grid()

Sanity check - print the difference between consecutive peak times, as well as the resulting estimate of the oscillation period (you may also want to look at a histogram of the time differences, or to compute their variance):

In [ ]:
print(f'time differences: {t_diff}\n')
print(f'period estimate (i.e., mean time difference): T = {T:.4f}')

Compute the moment of inertia about the $x$ axis (see formula from theory).

In [ ]:
J_x = 0.00 # <-- FIXME

# 4. Estimate the moment of inertia about the $y$-axis

Add cells here to repeat the same process as above...

In [ ]:
J_y = 0.00 # <-- FIXME

# 5. Estimate the moment of inertia about the $z$-axis

Add cells here to repeat the same process as above...

In [ ]:
J_z = 0.00 # <-- FIXME

# 6. Summarize and discuss the results

### Summary of results

In [ ]:
print(f'm = {m:.2} kg')
print(f'J_x = {J_x:.2e} kg m^2')
print(f'J_y = {J_y:.2e} kg m^2')
print(f'J_z = {J_z:.2e} kg m^2')

### Sources of error

**Modify the text in this cell** to discuss possible sources of error. For example:
* How uncertain was each measurement and each computed quantity?
* What assumptions were made and to what extent were these assumptions violated?
* Are there ways in which either the experiments or the analysis could be improved?

### Analysis of sensitivity

**Modify the text in this cell** to discuss the sensitivity of your results to possible sources of error. For example:
* Find the derivative of $J$ with respect to $m$. If you overestimate the mass, will you have overestimated or underestimated $J$?
* Find the derivative of $J$ with respect to $r$. If you overestimate the distance between the axis of rotation and the center of mass, will you have overestimated or underestimated $J$?
* Find the derivative of $J$ with respect to $T$. If you overestimate the period of oscillation, will you have overestimated or underestimated $J$?